# Reto: Análisis de Uso de TransMilenio en Domingos
### *by Juan Pablo Morales*

In [1053]:
import pandas as pd
import numpy as np

1. Se define la función `optimize_data_frame`, que se encargara de crear una copia optimizada de los cuatro data-frames para su posterior union, de la siguiente manera:
   - Convirtiendo columnas con pocos valores únicos en el tipo de dato `category`, de manera que ocupen menos espacio en memoria (con la función `categorize_cols`).
   > Algunas columnas requirieron volver a ser transformadas a categorias tras hacer el merge de los cuatro data-frames.
   - Removiendo columnas que no aporten información al data-frame, es decir aquellas cuyo valor sea nulo en todas las filas.
   - Convirtiendo las columnas de fechas y de valores numéricos al tipo de dato correcto (`datetime` y `float` respectivamente).
   - Removiendo filas de días diferentes a los del dataset.
   - Removiendo caracteres especiales y whitespaces de ciertas columnas.

> Tras algunos tests, el uso de memoria disminuye muy significativamente, para el data-frame final, que queda con unos `2,573,679` registros, el uso de memoria paso de +500 MB a ~164 MB

In [514]:
def categorize_cols(df:pd.DataFrame, cols:list):
    df[cols] = df[cols].astype("category")

In [901]:
def optimize_data_frame(df: pd.DataFrame):
    """This function will do the following optimizations to a copy of the provided data-frame `df`:
        - Set columns for date and numeric values to the right type
        - Remove useless columns/rows
        - Optimize cols with few unique values using `Series.astype()` method with `"category"` as the `dtype` parameter, significantly reducing memory consumption.

    Args:
        df (pd.DataFrame): data frame to be optimized and cleaned 

    Returns:
        pd.DataFrame: optimized copy of the original data-frame
    """
    optimized_df = df.copy()

    # Remove symbol at the start of "Linea" column's values
    optimized_df["Linea"] = optimized_df["Linea"].str.strip(" ()")

    # Optimize cols with few unique values using `Series.astype()` method with `"category"` as the `dtype` parameter, significantly reducing memory consumption.
    cols_to_categorize = [
        "Acceso_Estacion",
        "Day_Group_Type",
        "Dispositivo",
        "Emisor",
        "Estacion_Parada",
        "Fase",
        "Hora_Pico_SN",
        "Linea",
        "Nombre_Perfil",
        "Operador",
        "Sistema",
        "Tipo_Tarifa",
        "Tipo_Tarjeta"
    ]
    categorize_cols(optimized_df, cols_to_categorize)
    
    # Remove useless (completely empty) columns ["ID_Vehiculo","Ruta", "Tipo_Vehiculo"]
    optimized_df = optimized_df.dropna(axis="columns",how="all")

    # Set columns for date and numeric values to the right type
    optimized_df["Fecha_Transaccion"] = pd.to_datetime(df["Fecha_Transaccion"], format="%Y-%m-%d %H:%M:%S")
    optimized_df["Valor"] = df["Valor"].str.removeprefix("$ ").astype(float)

    # Remove rows where Fecha_Transaccion doesn't match Fecha_Clearing
    correct_data = optimized_df["Fecha_Transaccion"].dt.date == optimized_df["Fecha_Clearing"]
    optimized_df = optimized_df[correct_data]

    return optimized_df

In [904]:
def collect_data(data_list):
    """Method that groups all the data into a single, optimized data-frame

    Args:
        data_list (list): list of paths of csv files from which data is to be collected and grouped
    """
    final_df = pd.DataFrame()
    dfs = []

    for data in data_list: 
        
        print(f'Reading "{data}" ... ', end="")
        current_df = pd.read_csv(data, parse_dates=["Fecha_Clearing"], date_format="%Y-%m-%d")
        print("Done!")

        optimized_df = optimize_data_frame(current_df)
        dfs.append(optimized_df)
    
    # Set column "Es_Puente" as True In the first two data-frames
    for df in dfs[:2]:
        df["Es_Puente"] = True
        df["Es_Puente"] = df["Es_Puente"].astype("category")

    # Set column "Es_Puente" as False in the other two data-frames
    for df in dfs[2:]:
        df["Es_Puente"] = False
        df["Es_Puente"] = df["Es_Puente"].astype("category")

    # Perform an "union all" operation in all the df's
    final_df = pd.concat(dfs)

    # Add extra columns

    ## Add "Nombre_linea" column
    ### Retrieve and cleanup data-frame
    lineas_info = pd.read_csv("challenge/Lineas.csv").melt(var_name="Nombre_linea").drop(0).reset_index(drop=True)
    lineas_info = lineas_info.rename(columns={"value":"Linea"})
    lineas_info["Nombre_linea"] = lineas_info["Nombre_linea"].str.strip(" '")
    lineas_info["Linea"] = lineas_info["Linea"].str.strip(" '")

    ### Replace empty string values ('') with numpy's NaN datatype
    lineas_info.replace('',np.nan, inplace=True)
    lineas_info.dropna(inplace=True)

    ### Merge with final data-frame
    final_df = final_df.merge(lineas_info, how="left", on="Linea")

    ### Move new "Nombre_linea" column to be next to "Linea"
    linea_col_pos = final_df.columns.get_loc("Linea")
    linea_name_col = final_df.pop("Nombre_linea").astype("category")
    final_df.insert(linea_col_pos+1, "Nombre_linea", linea_name_col)


    ## Add "Nombre_emisor" column
    ### Retrieve and cleanup data-frame
    emisor_info = pd.read_json("challenge/emisor.json")
    emisor_info.columns = ["Emisor", "Nombre_emisor"]

    ### Merge with final data-frame
    final_df = final_df.merge(emisor_info, how="left", on="Emisor")

    ### Fill values that didn't have a match (NaN) with "Otras"
    final_df["Nombre_emisor"] = final_df["Nombre_emisor"].fillna("Otras")

    ### Move new "Nombre_emisor" column to be next to "Linea"
    emisor_col_pos = final_df.columns.get_loc("Emisor")
    emisor_name_col = final_df.pop("Nombre_emisor").astype("category")
    final_df.insert(emisor_col_pos+1, "Nombre_emisor", emisor_name_col)

    # Convert some cols to "category" datatype (again)
    cols_to_categorize = [
        "Acceso_Estacion",
        "Day_Group_Type",
        "Dispositivo",
        "Emisor",
        "Estacion_Parada",
        "Hora_Pico_SN",
        "Linea",
        "Es_Puente"
    ]
    categorize_cols(final_df, cols_to_categorize)

    return final_df

In [271]:
day1 = pd.read_csv("challenge/validacion20241020", parse_dates=["Fecha_Clearing"], date_format="%Y-%m-%d")

In [852]:
day1["Tipo_Tarifa"].duplicated(keep=False)
day1.nunique()

Acceso_Estacion                  401
Day_Group_Type                     2
Dispositivo                     1371
Emisor                             8
Estacion_Parada                  147
Fase                               1
Fecha_Clearing                     1
Fecha_Transaccion              64752
Hora_Pico_SN                       1
ID_Vehiculo                        0
Linea                             13
Nombre_Perfil                      7
Numero_Tarjeta                388048
Operador                           1
Ruta                               0
Saldo_Despues_Transaccion       8611
Saldo_Previo_a_Transaccion      8744
Sistema                            1
Tipo_Tarifa                        3
Tipo_Tarjeta                       2
Tipo_Vehiculo                      0
Valor                              3
dtype: int64

2. Se define una lista con los path de los data-frames, y se utilizan las funciones anteriormente definidas para optimizarlos, unirlos y obtener el data-frame final sobre el cual trabajaremos.

In [1055]:
# Read info of the four days from csv files and create a dataframe for each one
data_list = ["challenge/validacion20241020","challenge/validacion20241027","challenge/validacion20241103", "challenge/validacion20241110"]

# op_df = optimize_data_frame(day1)
op_df = collect_data(data_list)
op_df.info()

Reading "challenge/validacion20241020" ... Done!
Reading "challenge/validacion20241027" ... Done!
Reading "challenge/validacion20241103" ... Done!
Reading "challenge/validacion20241110" ... Done!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573679 entries, 0 to 2573678
Data columns (total 22 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   Acceso_Estacion             category      
 1   Day_Group_Type              category      
 2   Dispositivo                 category      
 3   Emisor                      category      
 4   Nombre_emisor               category      
 5   Estacion_Parada             category      
 6   Fase                        category      
 7   Fecha_Clearing              datetime64[ns]
 8   Fecha_Transaccion           datetime64[ns]
 9   Hora_Pico_SN                category      
 10  Linea                       category      
 11  Nombre_linea                category      
 12  Nombre_Perfil 

3. Se generan data-frames sumarizados que nos muestren la suma de los valores de la columna `Valor` por Fecha, Hora del día (1 a 24), Linea, Estación, Acceso, Emisor, Perfil.

3.1 Se define la función `format()` para darle formato de dinero a la columna Valor

In [1064]:
def to_currency(x):
    return "${:,.2f}".format(x)

### Valor por Fecha

In [ ]:
summary_by_date = op_df.pivot_table("Valor", index="Fecha_Clearing", aggfunc="sum")
summary_by_date["Valor"] = summary_by_date["Valor"].apply(to_currency)
summary_by_date

,Valor
Fecha_Clearing,
2024-10-20,"$1,680,940,000.00"
2024-10-27,"$1,747,398,150.00"
2024-11-03,"$1,745,490,000.00"
2024-11-10,"$1,578,308,850.00"


### Valor por hora del día

In [ ]:
horas = op_df["Fecha_Transaccion"].dt.hour.astype("category")
op_df.insert(op_df.columns.get_loc("Fecha_Transaccion")+1, "Hora_Transaccion", horas)

In [1070]:
summary_by_hour = op_df.pivot_table("Valor", index="Hora_Transaccion", aggfunc="sum", observed=False)
summary_by_hour["Valor"] = summary_by_hour["Valor"].apply(to_currency)
summary_by_hour

,Valor
Hora_Transaccion,
3,"$1,911,600.00"
4,"$87,407,500.00"
5,"$199,739,350.00"
6,"$329,564,550.00"
7,"$342,272,350.00"
8,"$369,653,150.00"
9,"$429,260,500.00"
10,"$462,453,600.00"
11,"$483,295,900.00"


### Valor por Linea

In [1077]:
summary_by_line = op_df.pivot_table("Valor", index="Nombre_linea", aggfunc="sum", observed=False)
summary_by_line["Valor"] = summary_by_line["Valor"].apply(to_currency)
summary_by_line

,Valor
Nombre_linea,
Zona A Caracas,"$704,275,950.00"
Zona B AutoNorte,"$963,794,150.00"
Zona C Av. Suba,"$501,143,300.00"
Zona D Calle 80,"$551,118,950.00"
Zona E NQS Central,"$478,564,300.00"
Zona F Av. Américas,"$686,029,050.00"
Zona F Calle 13,"$101,712,900.00"
Zona G NQS Sur,"$1,029,289,650.00"
Zona H Caracas Sur,"$674,844,300.00"


### Valor por Estación

In [1082]:
summary_by_station = op_df.pivot_table("Valor", index="Estacion_Parada", aggfunc="sum", observed=False)
summary_by_station["Valor"] = summary_by_station["Valor"].apply(to_currency)
summary_by_station

,Valor
Estacion_Parada,
(02000) Cabecera Autopista Norte,"$385,394,600.00"
(02001) Centro Comercial Santa Fe,"$29,895,200.00"
(02101) Toberin - Foundever,"$73,773,700.00"
(02102) Calle 161,"$12,108,500.00"
(02103) Mazurén,"$20,070,900.00"
...,...
(14004) Museo del Oro,"$48,833,850.00"
(14005) Las Aguas - Centro Colombo Americano,"$64,893,650.00"
(50003) Corral Molinos,"$1,782,550.00"


### Valor por Acceso

In [1083]:
summary_by_access = op_df.pivot_table("Valor", index="Acceso_Estacion", aggfunc="sum", observed=False)
summary_by_access["Valor"] = summary_by_access["Valor"].apply(to_currency)
summary_by_access

,Valor
Acceso_Estacion,
(00),"$790,950.00"
(01) ALIMENTACIÓN JUAN REY/ALTAMIRA/TIHUAQUE/LA RESURECCIÓN/VILLA DEL CERRO/PENINSULA,"$102,320,650.00"
(01) Acceso Alimentadores,"$86,700.00"
(01) Acceso Norte,"$379,068,400.00"
(01) Acceso Norte (Discapacidad),"$446,150.00"
...,...
(LA) PLATAFORMA 2 ALIMENTACION,"$52,893,950.00"
(LB) PLATAFORMA 2 DESALIMENTACIÓN - AURORA/AV. CARACAS/MARICHUELA/ALFONSO LOPEZ,"$661,400.00"
(LB) PLATAFORMA 2 DESALIMENTACIÓN OLARTE/BOSA LAURELES/AV. BOSA /TERMINAL/PERDOMO,"$3,581,500.00"


### Valor por Emisor

In [1084]:
summary_by_issuer = op_df.pivot_table("Valor", index="Nombre_emisor", aggfunc="sum", observed=False)
summary_by_issuer["Valor"] = summary_by_issuer["Valor"].apply(to_currency)
summary_by_issuer

,Valor
Nombre_emisor,
AV Villas,"$22,561,600.00"
Banco de Bogotá,"$14,145,250.00"
Bancolombia,"$209,627,000.00"
Bogota Card(Citizen),"$6,418,515,600.00"
Colpatria,"$46,497,900.00"
Otras,"$40,789,650.00"


### Valor por Perfil

In [1085]:
summary_by_profile = op_df.pivot_table("Valor", index="Nombre_Perfil", aggfunc="sum", observed=False)
summary_by_profile["Valor"] = summary_by_profile["Valor"].apply(to_currency)
summary_by_profile

,Valor
Nombre_Perfil,
(001) Adulto,"$2,626,355,500.00"
(001) Anonymous,"$2,990,600,850.00"
(002) Adulto Mayor,"$426,985,450.00"
(005) Discapacidad,"$130,870,850.00"
(006) Apoyo Ciudadano,"$245,983,400.00"
(009) Apoyo Ciudadano Reexpedición,"$147,045,600.00"
(101) Adulto PV,"$184,295,350.00"


### Data-frame sumarizado de todo

In [1105]:
# summarized_df = op_df.pivot_table("Valor", index=["Fecha_Clearing","Nombre_linea","Hora_Transaccion","Acceso_Estacion","Nombre_emisor"], aggfunc="sum", observed=False)#.sort_values(["Fecha_Clearing","Nombre_linea","Hora_Transaccion","Acceso_Estacion"])
summarized_df = op_df.groupby(["Fecha_Clearing","Nombre_linea","Hora_Transaccion","Acceso_Estacion","Nombre_emisor","Nombre_Perfil"], observed=False)
summarized_df.agg({"Valor":"sum"}).sample(20).sort_values(["Fecha_Clearing","Nombre_linea","Hora_Transaccion","Acceso_Estacion","Nombre_Perfil","Valor"], ascending=[True,True,True,True,True,False])

Valor
Fecha_Clearing Nombre_linea         Hora_Transaccion Acceso_Estacion                                    Nombre_emisor        Nombre_Perfil                            
2024-10-20     Zona B AutoNorte     19               (05) Acceso Suroriental                            Colpatria            (006) Apoyo Ciudadano                   0
               Zona C Av. Suba      14               (01) Acceso nor-oriente                            Otras                (002) Adulto Mayor                      0
               Zona H Caracas Sur   9                (10) Acceso Peatonal - Oriente                     AV Villas            (101) Adulto PV                         0
               Zona J Eje Ambiental 18               (03) BATERIA UNO VAGON ORIENTE CDS - CARRERA 32    Colpatria            (009) Apoyo Ciudadano Reexpedición      0
2024-10-27     Zona A Caracas       9                (02) Acceso nor-occidente                          AV Villas            (009) Apoyo Ciudadano Reexpedición      0
               Zona D Calle 80      4                (03) DESALIMENTACION JARDINES 2-2/B924 GUAYMARAL   Bancolombia          (001) Adulto                            0
               Zona E NQS Central   7                (BA) BATERIA VAGON ORIENTE PARQUE EL TUNAL         Colpatria            (001) Anonymous                         0
               Zona F Calle 13      13               (BA) BATERIA UNO VAGON SUR SANTA LUCÍA             Banco de Bogotá      (009) Apoyo Ciudadano Reexpedición      0
               Zona K Calle 26      3                (07) PLATAFORMA 1 DESALIMENTACION-CASA BLANCA/A... AV Villas            (006) Apoyo Ciudadano                   0
               Zona L Carrera 10    15               (01) BATERIA UNO VAGON NORTE ALQUERIA (Discapac... Colpatria            (006) Apoyo Ciudadano                   0
2024-11-03     Zona B AutoNorte     9                (08) Acceso Norte (Discapacidad)                   Banco de Bogotá      (006) Apoyo Ciudadano                   0
               Zona J Eje Ambiental 4                (01) BATERIA UNO VAGON NORTE SAN MARTIN (Discap... Banco de Bogotá      (002) Adulto Mayor                      0
2024-11-10     Zona C Av. Suba      7                (04) Acceso Sur (Discapacidad)                     Banco de Bogotá      (002) Adulto Mayor                      0
               Zona D Calle 80      17               (02) Acceso Peatonal                               Bogota Card(Citizen) (002) Adulto Mayor                      0
               Zona E NQS Central   4                (01) BATERIA UNO VAGON NORTE SANTA ISABEL (Disc... Bancolombia          (001) Adulto                            0
               Zona F Av. Américas  4                (08) PLATAFORMA 1 ALIMENTACIÓN - CASA BLANCA/AV... Colpatria            (006) Apoyo Ciudadano                   0
                                    23               (12) Plataforma 2 - Alimentación                   AV Villas            (005) Discapacidad                      0
               Zona G NQS Sur       22               (AA) BATERIA UNO VAGON NORTE ALCALÁ                AV Villas            (002) Adulto Mayor                      0
               Zona K Calle 26      5                (AA) BATERIA UNO VAGON NORTE HORTÚA                Banco de Bogotá      (001) Adulto                            0
               Zona L Carrera 10    22               (21) Acceso Túnel PLAT1 Alimentación               AV Villas            (006) Apoyo Ciudadano                   0

## 4. Análisis comparativo

### 4.1 Lineas más usadas

In [1115]:
op_df[["Linea","Nombre_linea"]].value_counts().sort_values(ascending=False)

Linea  Nombre_linea        
30     Zona G NQS Sur          391337
33     Zona B AutoNorte        342596
34     Zona H Caracas Sur      269591
31     Zona F Av. Américas     252065
36     Zona A Caracas          247466
12     Zona L Carrera 10       216742
35     Zona D Calle 80         207861
11     Zona K Calle 26         203076
32     Zona C Av. Suba         194012
38     Zona E NQS Central      173041
37     Zona J Eje Ambiental     39283
39     Zona F Calle 13          36591
Name: count, dtype: int64

### 4.2 Total pasajeros (puente vs. no puente)

In [1138]:
op_df["Es_Puente"].value_counts()

Es_Puente
True     1303269
False    1270410
Name: count, dtype: int64